In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
import requests
from bs4 import BeautifulSoup
# for i in range(2,10):

Product_name=[]
Prices=[]
Description=[]
Reviews=[]

for i in range(2,30):
    url = 'https://www.flipkart.com/search?q=phone%20under%2050000&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page='+str(i)
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
    }
                
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')
    boxes=soup.find_all("div",class_="DOjaWF gdgoEp")
    for box in boxes:
        #names
        names=box.find_all("div",class_="KzDlHZ")
        for i in names:
            name=i.text
            Product_name.append(name)
            
        #price
        prices=box.find_all("div",class_="Nx9bqj _4b5DiR")
        for i in prices:
            price=i.text
            Prices.append(price)
            
        #Description
        descript=box.find_all("ul",class_="G4BRas")
        for i in descript:
            desc=i.text
            Description.append(desc)
            
        #Reviews
        review=box.find_all("div",class_="XQDdHH")
        for i in review:
            rev=i.text
            Reviews.append(rev)

In [ ]:
Product_name=Product_name[:671]
Prices=Prices[:671]
Description=Description[:671]
Reviews=Reviews[:671]

In [ ]:
# Reviews=Reviews[]
print(len(Product_name))
print(len(Prices))
print(len(Description))
print(len(Reviews))

In [ ]:
df=pd.DataFrame({"Product_Name":Product_name,"Description":Description,"Rating":Reviews,"Price":Prices})
df.columns = ['Product Name', 'Description', 'Rating', 'Price']
df

In [ ]:
df.to_csv("Mobile_dataset.csv")

In [3]:
df = pd.read_csv("/Users/parthhathiwala/Downloads/PDS OEP/Mobile_dataset.csv", skipinitialspace=True)


In [5]:
df

,Unnamed: 0,Product Name,Description,Rating,Price
0,0,"SAMSUNG Galaxy S24 FE 5G (Blue, 128 GB)",8 GB RAM | 128 GB ROM17.02 cm (6.7 inch) Full ...,4.5,"₹39,799"
1,1,"Motorola Edge 50 Ultra 5G (Forest Grey, 512 GB)",12 GB RAM | 512 GB ROM17.02 cm (6.7 inch) Disp...,4.4,"₹49,999"
2,2,"realme P3 5G (Space Silver, 128 GB)",6 GB RAM | 128 GB ROM | Expandable Upto 2 TB16...,4.4,"₹16,999"
3,3,"REDMI Note 14 Pro 5G (Black, 128 GB)",8 GB RAM | 128 GB ROM16.94 cm (6.67 inch) Disp...,4.4,"₹23,999"
4,4,"Honor 200 Pro 5G (Ocean Cyan, 512 GB)",12 GB RAM | 512 GB ROM17.22 cm (6.78 inch) Ful...,4.1,"₹41,999"
...,...,...,...,...,...
666,666,"Google Pixel 9 (Wintergreen, 256 GB)",12 GB RAM | 256 GB ROM16.0 cm (6.3 inch) Displ...,4.2,"₹74,999"
667,667,"POCO C61 (Diamond Dust Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB17....,4.0,"₹6,600"
668,668,itel it2175 Pro | 2 inch Big Display | 1200 mA...,4 MB RAM | 4 MB ROM | Expandable Upto 32 GB5.0...,4.1,₹939
669,669,"SAMSUNG Galaxy A16 5G (Light Green, 128 GB)",6 GB RAM | 128 GB ROM17.02 cm (6.7 inch) Full ...,4.4,"₹15,499"


In [7]:
df[['Description']]

,Description
0,8 GB RAM | 128 GB ROM17.02 cm (6.7 inch) Full ...
1,12 GB RAM | 512 GB ROM17.02 cm (6.7 inch) Disp...
2,6 GB RAM | 128 GB ROM | Expandable Upto 2 TB16...
3,8 GB RAM | 128 GB ROM16.94 cm (6.67 inch) Disp...
4,12 GB RAM | 512 GB ROM17.22 cm (6.78 inch) Ful...
...,...
666,12 GB RAM | 256 GB ROM16.0 cm (6.3 inch) Displ...
667,4 GB RAM | 64 GB ROM | Expandable Upto 1 TB17....
668,4 MB RAM | 4 MB ROM | Expandable Upto 32 GB5.0...
669,6 GB RAM | 128 GB ROM17.02 cm (6.7 inch) Full ...


In [9]:
# Define functions to extract specific details from the Description column
def extract_ram(Description):
    if 'RAM' in Description:
        return Description.split('RAM')[0].strip().split()[-2] + ' ' + Description.split('RAM')[0].strip().split()[-1]
    return None

def extract_rom(Description):
    if 'ROM' in Description:
        return Description.split('ROM')[0].split('|')[-1].strip().split()[-2] + ' ' + Description.split('ROM')[0].strip().split()[-1]
    return None

def extract_battery(Description):
    if 'Battery' in Description:
        battery_info = Description.split('Battery')[0].strip().split()[-2] + ' ' + Description.split('Battery')[0].strip().split()[-1]
        # Remove "Camera" if it's present
        if 'Camera' in battery_info:
            battery_info = battery_info.replace('Camera', '').strip()
        return battery_info
    return None

def extract_display_size(description):
    match = re.search(r'([\d.]+)\s*cm', description)
    if match:
        cm = float(match.group(1))
        inches = cm / 2.54
        return f'{inches:.2f} inches'
    return None

def extract_warranty(Description):
    if 'Warranty' in Description:
        warranty_part = Description.split('Warranty')[0].strip()
        # Extract warranty info in terms of years and months
        warranty_match = re.search(r'(\d+ Year(?:s)?)? ?(\d+ Month(?:s)?)?', warranty_part)
        if warranty_match:
            years = warranty_match.group(1) or ''
            months = warranty_match.group(2) or ''
            return f'{years} {months}'.strip()
    return None

# Apply the extraction functions to create new columns
df['RAM'] = df['Description'].apply(extract_ram)
df['ROM'] = df['Description'].apply(extract_rom)
df['Battery'] = df['Description'].apply(extract_battery)
df['Display Size'] = df['Description'].apply(extract_display_size)
df['Warranty'] = df['Description'].apply(extract_warranty)

# Drop the original Description column
df.drop(columns=['Description'], inplace=True)


In [11]:
df

,Unnamed: 0,Product Name,Rating,Price,RAM,ROM,Battery,Display Size,Warranty
0,0,"SAMSUNG Galaxy S24 FE 5G (Blue, 128 GB)",4.5,"₹39,799",8 GB,128 GB,4700 mAh,6.70 inches,
1,1,"Motorola Edge 50 Ultra 5G (Forest Grey, 512 GB)",4.4,"₹49,999",12 GB,512 GB,4500 mAh,6.70 inches,None
2,2,"realme P3 5G (Space Silver, 128 GB)",4.4,"₹16,999",6 GB,128 GB,6000 mAh,6.67 inches,
3,3,"REDMI Note 14 Pro 5G (Black, 128 GB)",4.4,"₹23,999",8 GB,128 GB,5500 mAh,6.67 inches,
4,4,"Honor 200 Pro 5G (Ocean Cyan, 512 GB)",4.1,"₹41,999",12 GB,512 GB,5200 mAh,6.78 inches,None
...,...,...,...,...,...,...,...,...,...
666,666,"Google Pixel 9 (Wintergreen, 256 GB)",4.2,"₹74,999",12 GB,256 GB,4700 mAh,6.30 inches,
667,667,"POCO C61 (Diamond Dust Black, 64 GB)",4.0,"₹6,600",4 GB,64 GB,5000 mAh,6.71 inches,
668,668,itel it2175 Pro | 2 inch Big Display | 1200 mA...,4.1,₹939,4 MB,4 MB,0MP1200 mAh,2.00 inches,
669,669,"SAMSUNG Galaxy A16 5G (Light Green, 128 GB)",4.4,"₹15,499",6 GB,128 GB,5000 mAh,6.70 inches,


In [13]:
def clean_product_name(product_name):
    product_name = re.sub(r'\s*\(\d+\s*GB\)', '', product_name, flags=re.IGNORECASE)
    product_name = product_name.replace(' 5G', '')
    return product_name.strip()

# Create 'Type' column
df['Type'] = df['Product Name'].apply(lambda x: '5G' if '5G' in x else None)

# Clean 'Product Name' column
df['Product Name'] = df['Product Name'].apply(clean_product_name)

In [15]:
df.head(10)

,Unnamed: 0,Product Name,Rating,Price,RAM,ROM,Battery,Display Size,Warranty,Type
0,0,"SAMSUNG Galaxy S24 FE (Blue, 128 GB)",4.5,"₹39,799",8 GB,128 GB,4700 mAh,6.70 inches,,5G
1,1,"Motorola Edge 50 Ultra (Forest Grey, 512 GB)",4.4,"₹49,999",12 GB,512 GB,4500 mAh,6.70 inches,None,5G
2,2,"realme P3 (Space Silver, 128 GB)",4.4,"₹16,999",6 GB,128 GB,6000 mAh,6.67 inches,,5G
3,3,"REDMI Note 14 Pro (Black, 128 GB)",4.4,"₹23,999",8 GB,128 GB,5500 mAh,6.67 inches,,5G
4,4,"Honor 200 Pro (Ocean Cyan, 512 GB)",4.1,"₹41,999",12 GB,512 GB,5200 mAh,6.78 inches,None,5G
5,5,"SAMSUNG Galaxy S24 FE (Blue, 256 GB)",4.5,"₹45,950",8 GB,256 GB,4700 mAh,6.70 inches,,5G
6,6,"REDMI Note-14 Pro+ (Titan Black, 512 GB)",4.6,"₹37,999",12 GB,512 GB,6200 mAh,6.67 inches,,5G
7,7,"REDMI Note 14 Pro+ (Titan Black, 256 GB)",4.3,"₹31,999",8 GB,256 GB,6200 mAh,6.67 inches,,5G
8,8,"Honor 200 Pro (Black, 512 GB)",4.1,"₹41,999",12 GB,512 GB,5200 mAh,6.78 inches,None,5G
9,9,"SAMSUNG S23 (Phantom Black, 256 GB)",4.6,"₹49,999",8 GB,256 GB,Lithium Ion,6.10 inches,,5G


In [17]:
output_file_path = 'Processed_Mobile_dataset.csv'
df.to_csv(output_file_path, index=False)

In [19]:
df.drop('Warranty', axis=1, inplace=True)

In [21]:
df

,Unnamed: 0,Product Name,Rating,Price,RAM,ROM,Battery,Display Size,Type
0,0,"SAMSUNG Galaxy S24 FE (Blue, 128 GB)",4.5,"₹39,799",8 GB,128 GB,4700 mAh,6.70 inches,5G
1,1,"Motorola Edge 50 Ultra (Forest Grey, 512 GB)",4.4,"₹49,999",12 GB,512 GB,4500 mAh,6.70 inches,5G
2,2,"realme P3 (Space Silver, 128 GB)",4.4,"₹16,999",6 GB,128 GB,6000 mAh,6.67 inches,5G
3,3,"REDMI Note 14 Pro (Black, 128 GB)",4.4,"₹23,999",8 GB,128 GB,5500 mAh,6.67 inches,5G
4,4,"Honor 200 Pro (Ocean Cyan, 512 GB)",4.1,"₹41,999",12 GB,512 GB,5200 mAh,6.78 inches,5G
...,...,...,...,...,...,...,...,...,...
666,666,"Google Pixel 9 (Wintergreen, 256 GB)",4.2,"₹74,999",12 GB,256 GB,4700 mAh,6.30 inches,None
667,667,"POCO C61 (Diamond Dust Black, 64 GB)",4.0,"₹6,600",4 GB,64 GB,5000 mAh,6.71 inches,None
668,668,itel it2175 Pro | 2 inch Big Display | 1200 mA...,4.1,₹939,4 MB,4 MB,0MP1200 mAh,2.00 inches,None
669,669,"SAMSUNG Galaxy A16 (Light Green, 128 GB)",4.4,"₹15,499",6 GB,128 GB,5000 mAh,6.70 inches,5G


In [23]:
def remove_parentheses(product_name):
    return re.sub(r'\s*\(.*?\)', '', product_name)

df['Product Name'] = df['Product Name'].apply(remove_parentheses)

In [25]:
df

,Unnamed: 0,Product Name,Rating,Price,RAM,ROM,Battery,Display Size,Type
0,0,SAMSUNG Galaxy S24 FE,4.5,"₹39,799",8 GB,128 GB,4700 mAh,6.70 inches,5G
1,1,Motorola Edge 50 Ultra,4.4,"₹49,999",12 GB,512 GB,4500 mAh,6.70 inches,5G
2,2,realme P3,4.4,"₹16,999",6 GB,128 GB,6000 mAh,6.67 inches,5G
3,3,REDMI Note 14 Pro,4.4,"₹23,999",8 GB,128 GB,5500 mAh,6.67 inches,5G
4,4,Honor 200 Pro,4.1,"₹41,999",12 GB,512 GB,5200 mAh,6.78 inches,5G
...,...,...,...,...,...,...,...,...,...
666,666,Google Pixel 9,4.2,"₹74,999",12 GB,256 GB,4700 mAh,6.30 inches,None
667,667,POCO C61,4.0,"₹6,600",4 GB,64 GB,5000 mAh,6.71 inches,None
668,668,itel it2175 Pro | 2 inch Big Display | 1200 mA...,4.1,₹939,4 MB,4 MB,0MP1200 mAh,2.00 inches,None
669,669,SAMSUNG Galaxy A16,4.4,"₹15,499",6 GB,128 GB,5000 mAh,6.70 inches,5G


In [31]:
def extract_value_and_unit(column_name, unit):
    def extract(value):
        if pd.isna(value):
            return None
        value = str(value).replace('₹', '').replace(',', '')
        if unit in value:
            return value.replace(unit, '').strip()
        return value.strip()

    df[column_name] = df[column_name].astype(str).apply(extract)
    df.rename(columns={column_name: f'{column_name} ({unit})'}, inplace=True)

extract_value_and_unit('Price', '₹')
extract_value_and_unit('RAM', 'GB')
extract_value_and_unit('ROM', 'GB')
extract_value_and_unit('Battery', 'mAh')
extract_value_and_unit('Display Size', 'inches')

In [37]:
df

,Unnamed: 0,Product Name,Rating,Price (₹),RAM (GB),ROM (GB),Battery (mAh),Display Size (inches),Type
0,0,SAMSUNG Galaxy S24 FE,4.5,39799,8,128,4700,6.70,5G
1,1,Motorola Edge 50 Ultra,4.4,49999,12,512,4500,6.70,5G
2,2,realme P3,4.4,16999,6,128,6000,6.67,5G
3,3,REDMI Note 14 Pro,4.4,23999,8,128,5500,6.67,5G
4,4,Honor 200 Pro,4.1,41999,12,512,5200,6.78,5G
...,...,...,...,...,...,...,...,...,...
666,666,Google Pixel 9,4.2,74999,12,256,4700,6.30,None
667,667,POCO C61,4.0,6600,4,64,5000,6.71,None
668,668,itel it2175 Pro | 2 inch Big Display | 1200 mA...,4.1,939,4 MB,4 MB,0MP1200,2.00,None
669,669,SAMSUNG Galaxy A16,4.4,15499,6,128,5000,6.70,5G


In [39]:
df['Type'] = df['Type'].fillna('4G')

In [41]:
df

,Unnamed: 0,Product Name,Rating,Price (₹),RAM (GB),ROM (GB),Battery (mAh),Display Size (inches),Type
0,0,SAMSUNG Galaxy S24 FE,4.5,39799,8,128,4700,6.70,5G
1,1,Motorola Edge 50 Ultra,4.4,49999,12,512,4500,6.70,5G
2,2,realme P3,4.4,16999,6,128,6000,6.67,5G
3,3,REDMI Note 14 Pro,4.4,23999,8,128,5500,6.67,5G
4,4,Honor 200 Pro,4.1,41999,12,512,5200,6.78,5G
...,...,...,...,...,...,...,...,...,...
666,666,Google Pixel 9,4.2,74999,12,256,4700,6.30,4G
667,667,POCO C61,4.0,6600,4,64,5000,6.71,4G
668,668,itel it2175 Pro | 2 inch Big Display | 1200 mA...,4.1,939,4 MB,4 MB,0MP1200,2.00,4G
669,669,SAMSUNG Galaxy A16,4.4,15499,6,128,5000,6.70,5G


In [43]:
df = df.drop(df.columns[0], axis=1)

In [45]:
df

,Product Name,Rating,Price (₹),RAM (GB),ROM (GB),Battery (mAh),Display Size (inches),Type
0,SAMSUNG Galaxy S24 FE,4.5,39799,8,128,4700,6.70,5G
1,Motorola Edge 50 Ultra,4.4,49999,12,512,4500,6.70,5G
2,realme P3,4.4,16999,6,128,6000,6.67,5G
3,REDMI Note 14 Pro,4.4,23999,8,128,5500,6.67,5G
4,Honor 200 Pro,4.1,41999,12,512,5200,6.78,5G
...,...,...,...,...,...,...,...,...
666,Google Pixel 9,4.2,74999,12,256,4700,6.30,4G
667,POCO C61,4.0,6600,4,64,5000,6.71,4G
668,itel it2175 Pro | 2 inch Big Display | 1200 mA...,4.1,939,4 MB,4 MB,0MP1200,2.00,4G
669,SAMSUNG Galaxy A16,4.4,15499,6,128,5000,6.70,5G


In [47]:
output_file_path = 'Processed_Mobile_dataset_2.csv'
df.to_csv(output_file_path, index=False)

In [66]:
df.shape

(639, 8)

In [68]:
df.dtypes

Product Name              object
Rating                   float64
Price (₹)                 object
RAM (GB)                  object
ROM (GB)                  object
Battery (mAh)             object
Display Size (inches)     object
Type                      object
dtype: object

In [ ]:
'''df_cleaned = df[~df['RAM (GB)'].str.contains("MB", na=False)]
df_cleaned = df_cleaned[~df_cleaned['ROM (GB)'].str.contains("MB", na=False)]'''

In [72]:
df

,Product Name,Rating,Price (₹),RAM (GB),ROM (GB),Battery (mAh),Display Size (inches),Type
0,SAMSUNG Galaxy S24 FE,4.5,39799,8,128,4700,6.70,5G
1,Motorola Edge 50 Ultra,4.4,49999,12,512,4500,6.70,5G
2,realme P3,4.4,16999,6,128,6000,6.67,5G
3,REDMI Note 14 Pro,4.4,23999,8,128,5500,6.67,5G
4,Honor 200 Pro,4.1,41999,12,512,5200,6.78,5G
...,...,...,...,...,...,...,...,...
665,OnePlus 13,4.6,66299,12,256,6000,6.82,4G
666,Google Pixel 9,4.2,74999,12,256,4700,6.30,4G
667,POCO C61,4.0,6600,4,64,5000,6.71,4G
669,SAMSUNG Galaxy A16,4.4,15499,6,128,5000,6.70,5G


In [74]:
df.shape

(639, 8)

In [ ]:
'''df_cleaned = df[(df['RAM (GB)'] != 'None') & (df['ROM (GB)'] != 'None')]'''

In [80]:
df

,Product Name,Rating,Price (₹),RAM (GB),ROM (GB),Battery (mAh),Display Size (inches),Type
0,SAMSUNG Galaxy S24 FE,4.5,39799,8,128,4700,6.70,5G
1,Motorola Edge 50 Ultra,4.4,49999,12,512,4500,6.70,5G
2,realme P3,4.4,16999,6,128,6000,6.67,5G
3,REDMI Note 14 Pro,4.4,23999,8,128,5500,6.67,5G
4,Honor 200 Pro,4.1,41999,12,512,5200,6.78,5G
...,...,...,...,...,...,...,...,...
665,OnePlus 13,4.6,66299,12,256,6000,6.82,4G
666,Google Pixel 9,4.2,74999,12,256,4700,6.30,4G
667,POCO C61,4.0,6600,4,64,5000,6.71,4G
669,SAMSUNG Galaxy A16,4.4,15499,6,128,5000,6.70,5G


In [151]:
df.describe

<bound method NDFrame.describe of      Rating  RAM (GB)  ROM (GB)  Battery (mAh)  Display Size (inches)  Type  \
0       4.5         8       128         4700.0                   6.70     1   
1       4.4        12       512         4500.0                   6.70     1   
2       4.4         6       128         6000.0                   6.67     1   
3       4.4         8       128         5500.0                   6.67     1   
4       4.1        12       512         5200.0                   6.78     1   
..      ...       ...       ...            ...                    ...   ...   
665     4.6        12       256         6000.0                   6.82     0   
666     4.2        12       256         4700.0                   6.30     0   
667     4.0         4        64         5000.0                   6.71     0   
669     4.4         6       128         5000.0                   6.70     1   
670     4.2         8       128         6000.0                   6.72     1   

     Price_Label 

In [86]:
df

,Product Name,Rating,Price (₹),RAM (GB),ROM (GB),Battery (mAh),Display Size (inches),Type
0,SAMSUNG Galaxy S24 FE,4.5,39799,8,128,4700,6.70,5G
1,Motorola Edge 50 Ultra,4.4,49999,12,512,4500,6.70,5G
2,realme P3,4.4,16999,6,128,6000,6.67,5G
3,REDMI Note 14 Pro,4.4,23999,8,128,5500,6.67,5G
4,Honor 200 Pro,4.1,41999,12,512,5200,6.78,5G
...,...,...,...,...,...,...,...,...
665,OnePlus 13,4.6,66299,12,256,6000,6.82,4G
666,Google Pixel 9,4.2,74999,12,256,4700,6.30,4G
667,POCO C61,4.0,6600,4,64,5000,6.71,4G
669,SAMSUNG Galaxy A16,4.4,15499,6,128,5000,6.70,5G


In [98]:
print((df['RAM (GB)'] == 'None').sum())
print((df['ROM (GB)'] == 'None').sum())

60
1


In [ ]:
#  Clean and convert Price to numeric
df['Price (₹)'] = df['Price (₹)'].replace('[₹,]', '', regex=True)
df['Price (₹)'] = pd.to_numeric(df['Price (₹)'], errors='coerce')

#  Encode 'Type': 4G → 0, 5G → 1
df['Type'] = df['Type'].map({'4G': 0, '5G': 1})

#  Drop irrelevant or unusable columns
df.drop(columns=['Product Name'], inplace=True)

# Drop any remaining rows with missing values
df.dropna(inplace=True)

In [104]:
df.shape

(529, 7)

In [106]:
output_file_path = 'Processed_Mobile_dataset_4.csv'
df.to_csv(output_file_path, index=False)

In [112]:
df.dtypes

Rating                   float64
Price (₹)                  int64
RAM (GB)                   int64
ROM (GB)                   int64
Battery (mAh)            float64
Display Size (inches)    float64
Type                       int64
dtype: object

In [114]:
# Drop rows where RAM or ROM is 'None'
df = df[df['RAM (GB)'] != 'None']
df = df[df['ROM (GB)'] != 'None']

# Ensure columns are string type before using .str methods
df['RAM (GB)'] = df['RAM (GB)'].astype(str)
df['ROM (GB)'] = df['ROM (GB)'].astype(str)

# Drop rows where RAM or ROM contain 'MB'
df = df[~df['RAM (GB)'].str.contains('MB', na=False)]
df = df[~df['ROM (GB)'].str.contains('MB', na=False)]

In [118]:
df.shape

(529, 7)

In [120]:
df['Battery (mAh)'] = pd.to_numeric(df['Battery (mAh)'], errors='coerce')

# Calculate mean of valid numeric battery values
battery_mean = df['Battery (mAh)'].mean()

# Replace NaN values (i.e., non-numeric original entries) with the mean
df['Battery (mAh)'].fillna(battery_mean, inplace=True)

/var/folders/hs/t8k2q10j6gb7832fcz6922f00000gn/T/ipykernel_32339/3742643316.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Battery (mAh)'].fillna(battery_mean, inplace=True)


In [122]:
df.head(10)

,Rating,Price (₹),RAM (GB),ROM (GB),Battery (mAh),Display Size (inches),Type
0,4.5,39799,8,128,4700.0,6.70,1
1,4.4,49999,12,512,4500.0,6.70,1
2,4.4,16999,6,128,6000.0,6.67,1
3,4.4,23999,8,128,5500.0,6.67,1
4,4.1,41999,12,512,5200.0,6.78,1
5,4.5,45950,8,256,4700.0,6.70,1
6,4.6,37999,12,512,6200.0,6.67,1
7,4.3,31999,8,256,6200.0,6.67,1
8,4.1,41999,12,512,5200.0,6.78,1
10,4.5,46500,8,256,4700.0,6.70,1


In [124]:
output_file_path = 'Processed_Mobile_dataset_5.csv'
df.to_csv(output_file_path, index=False)

In [126]:
df.dtypes

Rating                   float64
Price (₹)                  int64
RAM (GB)                  object
ROM (GB)                  object
Battery (mAh)            float64
Display Size (inches)    float64
Type                       int64
dtype: object

In [130]:
df_f = pd.read_csv("Processed_Mobile_dataset_5.csv")

In [132]:
df_f

,Rating,Price (₹),RAM (GB),ROM (GB),Battery (mAh),Display Size (inches),Type
0,4.5,39799,8,128,4700.0,6.70,1
1,4.4,49999,12,512,4500.0,6.70,1
2,4.4,16999,6,128,6000.0,6.67,1
3,4.4,23999,8,128,5500.0,6.67,1
4,4.1,41999,12,512,5200.0,6.78,1
...,...,...,...,...,...,...,...
524,4.6,66299,12,256,6000.0,6.82,0
525,4.2,74999,12,256,4700.0,6.30,0
526,4.0,6600,4,64,5000.0,6.71,0
527,4.4,15499,6,128,5000.0,6.70,1
